# Transaction Fraud Detectation Project -- De-duplication

The 'card transactions.csv' is a very big dataset, if I try to conduct every process all in one Jupyter Notebook, it will cause my kernel to die. So I separate the steps into several notebooks and deal with the process one by one. And this notebook is for <b>de-duplication</b>.

I found that the feature engineering part may create duplicate features, which will affect model predictions. Here is the code to remove duplicates from the generated features.

In [1]:
import pandas as pd 
import numpy as np
import datetime as dt
import time
import glob
start_time = dt.datetime.now()
%matplotlib inline

In [2]:
joined_files_3_ = glob.glob('output/3_3_*.csv')
joined_files_3_

['output/3_3_8_counts_entities_03_Merch_description.csv',
 'output/3_3_8_counts_entities_14_Card_Merch_des_zip.csv',
 'output/3_3_8_counts_entities_01_Cardnum.csv',
 'output/3_3_3_zip3_velocity_counts_and_days_since.csv',
 'output/3_3_7_zip3_variability.csv',
 'output/3_3_8_counts_entities_07_Card_Merch_description.csv',
 'output/3_3_8_counts_entities_17_Card_Merch_num_des_zip.csv',
 'output/3_3_7_variability.csv',
 'output/3_3_8_counts_entities_08_Card_Merch_state.csv',
 'output/3_3_8_counts_entities_19_Card_Merch_des_state_zip.csv',
 'output/3_3_1_bf_law.csv',
 'output/3_3_5_relative&daysince_vel.csv',
 'output/3_3_3_velocity_counts_and_days_since.csv',
 'output/3_3_8_counts_entities_18_Card_Merch_num_state_zip.csv',
 'output/3_3_8_counts_entities_05_Merch_all_info.csv',
 'output/3_3_8_counts_entities_09_Card_Merch_zip.csv',
 'output/3_3_5_zip3_relative&daysince_vel.csv',
 'output/3_3_8_counts_entities_12_Card_Merch_num_zip.csv',
 'output/3_3_8_counts_entities_15_Card_Merch_state_zip

In [3]:
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []
    for t, v in groups.items():
        dcols = frame[v].to_dict(orient="list")

        vs = list(dcols.values())
        ks = list(dcols.keys())
        lvs = len(vs)

        for i in range(1, lvs+1):
            for j in range(i+1,lvs+1):
                if vs[-i] == vs[-j]: 
                    dups.append(ks[-i])
                    break

    return dups

In [4]:
numvar = {}
listvar = []

def num_column_count(dataframe, name):
    
    global numvar, listvar
    
    numvars_new = len(dataframe.columns) - 2 # excluded 2 are record and fraud_label
    numvar[name] = numvars_new
    
    listvar_new = dataframe.columns.to_list()[2:] # excluded 2 are record and fraud_label
    listvar += listvar_new

    print("from Dataframe: " + name)
    print("new variables:", numvars_new,"  total variables:", sum(numvar.values()))

In [8]:
%%time
begin=time.time()

number_of_file = len(joined_files_3_)

for num_for_dedup in range(1, number_of_file + 1):
    
    st=time.time()

    vars_prep_for_dedup = pd.read_csv(joined_files_3_[-num_for_dedup])
    new_filepath = joined_files_3_[-num_for_dedup].replace("output", "output_dedup").replace(".csv", "_dedup.csv")
   
    
    for num_before in range(1 + num_for_dedup, number_of_file + 1):
        
        df_concated = pd.concat([pd.read_csv(joined_files_3_[-num_before]).iloc[:,2:],
                                 vars_prep_for_dedup], axis = 1)
        
        dup_columns_name = duplicate_columns(df_concated)
        del df_concated
        
        vars_prep_for_dedup.drop(dup_columns_name, axis=1, inplace=True, errors='ignore')
        
    vars_prep_for_dedup.to_csv(new_filepath, index=False)
    
    new_filename = new_filepath.replace("_dedup.csv", "").split('/')[1] 
    num_column_count(vars_prep_for_dedup, new_filename)
    
    del vars_prep_for_dedup
    print(f'Run time for {new_filename}--------{time.time()-st:0.2f}s', end='\n\n')

print(f'Total run time:{(time.time()-begin)/60:0.2f}min')

/var/folders/_l/blrtgl294gl19nj02vfrwr180000gn/T/ipykernel_11836/1715190757.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dcols = frame[v].to_dict(orient="list")


from Dataframe: 3_3_8_counts_entities_06_Card_Merchnum
new variables: 35   total variables: 35
Run time for 3_3_8_counts_entities_06_Card_Merchnum--------86.85s



/var/folders/_l/blrtgl294gl19nj02vfrwr180000gn/T/ipykernel_11836/1715190757.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dcols = frame[v].to_dict(orient="list")


from Dataframe: 3_3_8_counts_entities_04_Merch_num_des
new variables: 23   total variables: 58
Run time for 3_3_8_counts_entities_04_Merch_num_des--------76.08s



/var/folders/_l/blrtgl294gl19nj02vfrwr180000gn/T/ipykernel_11836/1715190757.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dcols = frame[v].to_dict(orient="list")
/var/folders/_l/blrtgl294gl19nj02vfrwr180000gn/T/ipykernel_11836/1715190757.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dcols = frame[v].to_dict(orient="list")
/var/folders/_l/blrtgl294gl19nj02vfrwr180000gn/T/ipykernel_11836/1715190757.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dcols = frame[v].to_dict(orient="list")


...

from Dataframe: 3_3_8_counts_entities_18_Card_Merch_num_state_zip
new variables: 6   total variables: 1701
Run time for 3_3_8_counts_entities_18_Card_Merch_num_state_zip--------27.85s



/var/folders/_l/blrtgl294gl19nj02vfrwr180000gn/T/ipykernel_11836/1715190757.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dcols = frame[v].to_dict(orient="list")


from Dataframe: 3_3_3_velocity_counts_and_days_since
new variables: 207   total variables: 1908
Run time for 3_3_3_velocity_counts_and_days_since--------37.96s



/var/folders/_l/blrtgl294gl19nj02vfrwr180000gn/T/ipykernel_11836/1715190757.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dcols = frame[v].to_dict(orient="list")


from Dataframe: 3_3_5_relative&daysince_vel
new variables: 244   total variables: 2152
Run time for 3_3_5_relative&daysince_vel--------57.29s

from Dataframe: 3_3_1_bf_law
new variables: 39   total variables: 2191
Run time for 3_3_1_bf_law--------22.54s

from Dataframe: 3_3_8_counts_entities_19_Card_Merch_des_state_zip
new variables: 3   total variables: 2194
Run time for 3_3_8_counts_entities_19_Card_Merch_des_state_zip--------15.79s

from Dataframe: 3_3_8_counts_entities_08_Card_Merch_state
new variables: 42   total variables: 2236
Run time for 3_3_8_counts_entities_08_Card_Merch_state--------16.79s

from Dataframe: 3_3_7_variability
new variables: 526   total variables: 2762
Run time for 3_3_7_variability--------68.21s

from Dataframe: 3_3_8_counts_entities_17_Card_Merch_num_des_zip
new variables: 1   total variables: 2763
Run time for 3_3_8_counts_entities_17_Card_Merch_num_des_zip--------4.86s

from Dataframe: 3_3_8_counts_entities_07_Card_Merch_description
new variables: 32   tot

In [9]:
df_numvar = pd.DataFrame(numvar, index=["count_variables"]).T.iloc[::-1].reset_index()
df_numvar.loc['Total'] = df_numvar.sum(numeric_only=True)
df_numvar.columns = ['filename', 'count_variables']

df_numvar.to_csv('3_count_variables_dedup.csv')
df_numvar

,filename,count_variables
0,3_3_8_counts_entities_03_Merch_description,114.0
1,3_3_8_counts_entities_14_Card_Merch_des_zip,10.0
2,3_3_8_counts_entities_01_Cardnum,80.0
3,3_3_3_zip3_velocity_counts_and_days_since,7.0
4,3_3_7_zip3_variability,18.0
5,3_3_8_counts_entities_07_Card_Merch_description,32.0
6,3_3_8_counts_entities_17_Card_Merch_num_des_zip,1.0
7,3_3_7_variability,526.0
8,3_3_8_counts_entities_08_Card_Merch_state,42.0
9,3_3_8_counts_entities_19_Card_Merch_des_state_zip,3.0


In [10]:
df_listvar = pd.DataFrame(listvar, columns = ["variables"]).drop_duplicates().iloc[::-1].reset_index(drop = True)

df_listvar.to_csv('output_dedup/3_list_variables_dedup.csv')
df_listvar

,variables
0,Merch_description_unique_count_for_Card_Merch_...
1,Merch_description_unique_count_for_Card_Merch_...
2,Merch_description_unique_count_for_Card_Merch_...
3,Merch_description_unique_count_for_Card_Merch_...
4,Merch_description_unique_count_for_Card_Merch_...
...,...
3017,Card_Merchnum_unique_count_for_Merch_descripti...
3018,Card_Merchnum_unique_count_for_Merch_descripti...
3019,Card_Merchnum_unique_count_for_Merch_descripti...
3020,Card_Merchnum_unique_count_for_Merch_descripti...


In [11]:
bf_law = pd.read_csv('output_dedup/3_3_1_bf_law_dedup.csv')

In [12]:
# I didn't remove the attributes from my CSV file, they are in '3_3_1_bf_law.csv'
# Because the attributes are unique, I can't remove them by de-duplicate codes
# I'll manually remove them here by slicing the data frame
bf_law = bf_law.iloc[:,[1,10,12,39,40]]

In [13]:
bf_law.to_csv('output_dedup/3_3_1_bf_law_dedup_remove_attri.csv', index = False)

In [14]:
print('real total num of variables: {}.'.format(df_numvar.iloc[-1,1] - (41-5))) # 41 is the # of col in '3_3_1_bf_law_dedup.csv'

real total num of variables: 2988.0.


In [15]:
print('duration: ',dt.datetime.now() - start_time)

duration:  0:48:25.629342
